In [1]:
from model import Posicao, Extrato
from views import FundoInvestimento, Acao, FundoImobiliario
#from controllers import MainController
import numpy as np
import pandas as pd 
from datetime import datetime

import plotly.graph_objs as go

# Config
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-1-71a943eb1a53>:15: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
posicao_model = Posicao() 
posicao_model.load_data()
extrato = Extrato(2010, 2020)
fundo_investimento = FundoInvestimento(
    posicao=posicao_model.fis, 
    extrato=extrato.extrato_fis
)
acoes = Acao(posicao=posicao_model.acoes, extrato=extrato.df)
fiis = FundoImobiliario(posicao=posicao_model.fiis, extrato=extrato.df)

/Users/gabriellopes/Documents/finance-app/app/classes/model.py:286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.aportes_fi_hist['Nome'] = self.aportes_fi_hist['Descricao'].apply(self.__map_fi)
/Users/gabriellopes/Documents/finance-app/app/classes/model.py:295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.ir_fi_hist['Nome'] = self.ir_fi_hist['Descricao'].apply(self.__map_fi)


In [3]:
#fundo_investimento.extrato[fundo_investimento.extrato['Nome'].str.contains('unknown')].head()
extrato.aportes_fi_hist[extrato.aportes_fi_hist['Nome'] == 'unknown']

,Liq,Mov,Descricao,Valor,Saldo,ano,mes,Nome


In [4]:
extrato.aportes_fi_hist['Nome'].unique()

array(['Equitas Selection FIC FIA', 'Polo Norte I FIC FIM',
       'XP Macro FIM', 'Bahia AM Maraú FIC de FIM',
       'AZ Quest Multi FIC FIM', 'Mauá Macro FIC FIM',
       'XP Long Short FIC FIM', 'Mirae Asset Multimercado ',
       'XP GlobCredit FICFIM', 'FICFIVot. CambDola',
       'BNP Paribas Inflação FI RF', 'Hedge Fic Fim',
       'Azul Quantitativo FIM', 'ABSOLUTO CONSUMO', 'Legan Low Vol FIM',
       'XP MULT-INV FIC FIA', 'Icatu Vanguarda Pré-Fixado FIRF '],
      dtype=object)

In [5]:
fundo_investimento.posicao_hist[fundo_investimento.posicao_hist['Nome'] == 'AZ Quest Multi FIC FIM']\
    .sort_values('data_posicao', ascending=False).head()

,Nome,Data,Qtd Cotas,Valor Cota,Valor Bruto,IR,IOF,Valor Liquido,Aplicacao Pendente,Total Bruto,period,mes,ano,data_posicao
2,AZ Quest Multi FIC FIM,2020-02-28,"1,464.80",12.95,"18,967.40",93.33,0,"18,874.07",0,"18,967.40",2020/2,2,2020,2020-02-28
199,AZ Quest Multi FIC FIM,2020-01-31,"1,464.80",12.84,"18,801.04",68.38,0,"18,732.66",0,"18,801.04",2020/1,1,2020,2020-01-31
164,AZ Quest Multi FIC FIM,2019-12-31,"1,464.80",12.74,"18,661.13",47.40,0,"18,613.73",0,"18,661.13",2019/12,12,2019,2019-12-31
222,AZ Quest Multi FIC FIM,2019-11-29,"1,464.80",12.52,"18,345.13",0.00,0,"18,345.13",0,"18,345.13",2019/11,11,2019,2019-11-29
192,AZ Quest Multi FIC FIM,2019-10-31,"1,476.76",12.54,"18,525.29",154.39,0,"18,370.90",0,"18,525.29",2019/10,10,2019,2019-10-31


In [6]:
df_pos = fundo_investimento.calcula_rentabilidade(2010, 2020)

df_pos['Data'] = df_pos['Data'].astype(np.datetime64)

start = datetime(2010, 1, 1)
end = datetime(int(datetime.now().year), 12, 31)
ix_date = pd.date_range(start, end, freq='M')
df_ixDate = pd.DataFrame(ix_date, columns=['Data'])
df_ixDate['ano'] = df_ixDate['Data'].dt.year
df_ixDate['mes'] = df_ixDate['Data'].dt.month

df_tmp = pd.DataFrame([], columns=df_pos.columns)

for fi in df_pos['Nome'].unique():
    df_pos_ = df_ixDate.merge(df_pos[df_pos['Nome'] ==  fi], 
                                    how='left', 
                                    left_on=['ano', 'mes'], 
                                    right_on=['ano', 'mes']).reset_index().fillna(0)
    df_pos_['Nome'] = fi
    df_pos_['mes'] = df_pos_['Data_x'].dt.month
    df_pos_['ano'] = df_pos_['Data_x'].dt.year
    df_pos_['data_posicao'] = df_pos_['Data_x']
    df_tmp = df_tmp.append(df_pos_)


display(df_tmp[df_tmp['Nome'] == 'AZ Quest Multi FIC FIM'])

,Nome,Data,Qtd Cotas,Valor Cota,Valor Bruto,IR,IOF,Valor Liquido,Aplicacao Pendente,Total Bruto,period,mes,ano,data_posicao,index,rendimento,periodo_cont,rendimento_acum,level_0,Data_x,Data_y
0,AZ Quest Multi FIC FIM,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0,1,2010,2010-01-31,0.00,0.00,0.00,0.00,0.00,2010-01-31,0
1,AZ Quest Multi FIC FIM,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0,2,2010,2010-02-28,0.00,0.00,0.00,0.00,1.00,2010-02-28,0
2,AZ Quest Multi FIC FIM,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0,3,2010,2010-03-31,0.00,0.00,0.00,0.00,2.00,2010-03-31,0
3,AZ Quest Multi FIC FIM,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0,4,2010,2010-04-30,0.00,0.00,0.00,0.00,3.00,2010-04-30,0
4,AZ Quest Multi FIC FIM,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0,5,2010,2010-05-31,0.00,0.00,0.00,0.00,4.00,2010-05-31,0
5,AZ Quest Multi FIC FIM,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0,6,2010,2010-06-30,0.00,0.00,0.00,0.00,5.00,2010-06-30,0
6,AZ Quest Multi FIC FIM,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0,7,2010,2010-07-31,0.00,0.00,0.00,0.00,6.00,2010-07-31,0
7,AZ Quest Multi FIC FIM,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0,8,2010,2010-08-31,0.00,0.00,0.00,0.00,7.00,2010-08-31,0
8,AZ Quest Multi FIC FIM,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0,9,2010,2010-09-30,0.00,0.00,0.00,0.00,8.00,2010-09-30,0
9,AZ Quest Multi FIC FIM,NaN,0.00,0.00,0.00,0.00,0.00,0.00,0,0.00,0,10,2010,2010-10-31,0.00,0.00,0.00,0.00,9.00,2010-10-31,0


In [7]:
resumo_fi = fundo_investimento.resumo_novo(2010, 2020)

In [8]:
#display(resumo_fi[resumo_fi['Nome'] == 'Icatu Vanguarda Pré-Fixado FIRF '].sum())

display(resumo_fi[resumo_fi['Nome'] == 'Icatu Vanguarda Pré-Fixado FIRF '])

,Nome,ano,mes,periodo_cont,data_posicao,Total Bruto,aporte,retirada,rendimento_resgatado,rendimento_posicao
660,Icatu Vanguarda Pré-Fixado FIRF,2010,1,0,2010-01-31,0.00,0.00,0.00,0.00,0.00
661,Icatu Vanguarda Pré-Fixado FIRF,2010,2,0,2010-02-28,0.00,0.00,0.00,0.00,0.00
662,Icatu Vanguarda Pré-Fixado FIRF,2010,3,0,2010-03-31,0.00,0.00,0.00,0.00,0.00
663,Icatu Vanguarda Pré-Fixado FIRF,2010,4,0,2010-04-30,0.00,0.00,0.00,0.00,0.00
664,Icatu Vanguarda Pré-Fixado FIRF,2010,5,0,2010-05-31,0.00,0.00,0.00,0.00,0.00
665,Icatu Vanguarda Pré-Fixado FIRF,2010,6,0,2010-06-30,0.00,0.00,0.00,0.00,0.00
666,Icatu Vanguarda Pré-Fixado FIRF,2010,7,0,2010-07-31,0.00,0.00,0.00,0.00,0.00
667,Icatu Vanguarda Pré-Fixado FIRF,2010,8,0,2010-08-31,0.00,0.00,0.00,0.00,0.00
668,Icatu Vanguarda Pré-Fixado FIRF,2010,9,0,2010-09-30,0.00,0.00,0.00,0.00,0.00
669,Icatu Vanguarda Pré-Fixado FIRF,2010,10,0,2010-10-31,0.00,0.00,0.00,0.00,0.00


In [9]:
start = datetime(resumo_fi['ano'].min(), 1, 1)
end = datetime(resumo_fi['ano'].max(), 12, 1)

print('De: {} até {}'.format(start, end))

De: 2010-01-01 00:00:00 até 2020-12-01 00:00:00


In [10]:
resumo_fi['Nome'].unique()

array(['AZ Quest Multi FIC FIM', 'Azul Quantitativo FIM',
       'BNP Paribas Inflação FI RF', 'Bahia AM Maraú FIC de FIM',
       'Equitas Selection FIC FIA', 'Icatu Vanguarda Pré-Fixado FIRF ',
       'Mauá Macro FIC FIM', 'Mirae Asset Multimercado ',
       'NP Hedge FIC FIM', 'Polo Norte I FIC FIM',
       'XP Absoluto Consumo FIA', 'XP Long Short FIC FIM', 'XP Macro FIM'],
      dtype=object)

In [11]:
fundo_investimento.extrato.head()

,Nome,ano,mes,Vlr Aporte,Vlr Resgate,Vlr IR,Rendimento Resgatado
0,ABSOLUTO CONSUMO,2013,1,"1,700.00",0.00,0.00,0.00
1,ABSOLUTO CONSUMO,2013,5,700.00,0.00,0.00,0.00
2,AZ Quest Multi FIC FIM,2017,3,"15,000.00",0.00,0.00,0.00
3,Azul Quantitativo FIM,2013,6,"5,000.00",0.00,0.00,0.00
4,Azul Quantitativo FIM,2013,8,"10,200.00",0.00,0.00,0.00


In [12]:
display(resumo_fi.sort_values(['ano', 'mes'], ascending=False).head(2))
resumo_fi[resumo_fi['Nome'] == 'Azul Quantitativo FIM']

,Nome,ano,mes,periodo_cont,data_posicao,Total Bruto,aporte,retirada,rendimento_resgatado,rendimento_posicao
131,AZ Quest Multi FIC FIM,2020,12,0,2020-12-31,0.00,0.00,0.00,0.00,0.00
263,Azul Quantitativo FIM,2020,12,0,2020-12-31,0.00,0.00,0.00,0.00,0.00


,Nome,ano,mes,periodo_cont,data_posicao,Total Bruto,aporte,retirada,rendimento_resgatado,rendimento_posicao
132,Azul Quantitativo FIM,2010,1,0,2010-01-31,0.00,0.00,0.00,0.00,0.00
133,Azul Quantitativo FIM,2010,2,0,2010-02-28,0.00,0.00,0.00,0.00,0.00
134,Azul Quantitativo FIM,2010,3,0,2010-03-31,0.00,0.00,0.00,0.00,0.00
135,Azul Quantitativo FIM,2010,4,0,2010-04-30,0.00,0.00,0.00,0.00,0.00
136,Azul Quantitativo FIM,2010,5,0,2010-05-31,0.00,0.00,0.00,0.00,0.00
137,Azul Quantitativo FIM,2010,6,0,2010-06-30,0.00,0.00,0.00,0.00,0.00
138,Azul Quantitativo FIM,2010,7,0,2010-07-31,0.00,0.00,0.00,0.00,0.00
139,Azul Quantitativo FIM,2010,8,0,2010-08-31,0.00,0.00,0.00,0.00,0.00
140,Azul Quantitativo FIM,2010,9,0,2010-09-30,0.00,0.00,0.00,0.00,0.00
141,Azul Quantitativo FIM,2010,10,0,2010-10-31,0.00,0.00,0.00,0.00,0.00


In [13]:
df_pos[df_pos['Nome'] == 'Bahia AM Maraú FIC de FIM']

,Nome,Data,Qtd Cotas,Valor Cota,Valor Bruto,IR,IOF,Valor Liquido,Aplicacao Pendente,Total Bruto,period,mes,ano,data_posicao,index,rendimento,periodo_cont,rendimento_acum
0,Bahia AM Maraú FIC de FIM,2017-10-31,"34,777.00",2.10,"72,862.11",0.00,0,"72,862.11",0,"72,862.11",2017/10,10,2017,2017-10-31,77.00,0.00,1.00,0.00
1,Bahia AM Maraú FIC de FIM,2017-11-30,"34,765.76",2.10,"73,107.49",11.83,0,"73,095.66",0,"73,107.49",2017/11,11,2017,2017-11-30,61.00,245.38,2.00,245.38
2,Bahia AM Maraú FIC de FIM,2017-12-29,"34,765.76",2.13,"74,127.79",235.41,0,"73,892.38",0,"74,127.79",2017/12,12,2017,2017-12-28,70.00,"1,020.30",3.00,"1,265.68"
3,Bahia AM Maraú FIC de FIM,2018-01-31,"34,765.76",2.17,"75,332.65",506.50,0,"74,826.15",0,"75,332.65",2018/1,1,2018,2018-01-31,147.00,"1,204.86",4.00,"2,470.54"
4,Bahia AM Maraú FIC de FIM,2018-02-28,"34,765.76",2.21,"76,729.76",820.85,0,"75,908.91",0,"76,729.76",2018/2,2,2018,2018-02-28,128.00,"1,397.11",5.00,"3,867.65"
5,Bahia AM Maraú FIC de FIM,2018-03-29,"34,765.76",2.26,"78,513.46","1,222.19",0,"77,291.27",0,"78,513.46",2018/3,3,2018,2018-03-29,188.00,"1,783.70",6.00,"5,651.35"
6,Bahia AM Maraú FIC de FIM,2018-04-30,"34,765.76",2.27,"79,075.62","1,196.18",0,"77,879.44",0,"79,075.62",2018/4,4,2018,2018-04-30,206.00,562.16,7.00,"6,213.51"
7,Bahia AM Maraú FIC de FIM,2018-05-30,"34,399.25",2.26,"77,833.42",284.32,0,"77,549.10",0,"77,833.42",2018/5,5,2018,2018-05-30,182.00,"-1,242.20",8.00,"4,971.31"
8,Bahia AM Maraú FIC de FIM,2018-06-29,"34,399.25",2.27,"78,112.39",340.10,0,"77,772.29",0,"78,112.39",2018/6,6,2018,2018-06-29,151.00,278.97,9.00,"5,250.28"
9,Bahia AM Maraú FIC de FIM,2018-07-31,"34,399.25",2.29,"78,811.37",479.90,0,"78,331.47",0,"78,811.37",2018/7,7,2018,2018-07-31,229.00,698.98,10.00,"5,949.26"


In [14]:
extrato.aportes_xp

,Liq,Mov,Descricao,Valor,Saldo,ano,mes
3,2020-05-28,2020-05-28,TED - RECEBIMENTO DE TED - SPB,"7,000.00","8,478.00",2020,5
25,2020-03-13,2020-03-13,TED - RECEBIMENTO DE TED - SPB,"5,000.00","5,509.66",2020,3
36,2020-01-31,2020-01-31,TED - RECEBIMENTO DE TED - SPB,"12,000.00","13,352.61",2020,1
40,2020-01-21,2020-01-21,TED - RECEBIMENTO DE TED - SPB,"1,000.00","12,186.54",2020,1
44,2019-12-23,2019-12-23,TED - RECEBIMENTO DE TED - SPB,"11,000.00","11,156.82",2019,12
64,2019-09-12,2019-09-12,TED - RECEBIMENTO DE TED - SPB,"47,000.00","47,547.52",2019,9
91,2018-12-17,2018-12-17,TED - RECEBIMENTO DE TED - SPB,"30,000.00","30,305.18",2018,12
144,2017-10-05,2017-10-05,TED - RECEBIMENTO DE TED - SPB,"73,000.00","73,259.50",2017,10
189,2017-05-24,2017-05-24,TED - RECEBIMENTO DE TED - SPB,100.00,250.89,2017,5
198,2017-03-17,2017-03-17,TED - RECEBIMENTO DE TED - SPB,"30,000.00","30,180.67",2017,3
